# Pattern Matching Cookbook

## Common matching operations
If we're matching a literal (`string`, `int`, `float`, `None`, `bool`) then the literal can be directly used in a `case` arm. 

The example below shows off some of the ways we can match. 
I most frequently use `unions` (`|`), `guards`, `capture` or `wildcard` (`_`) expressions.

You may be confused by `int()` syntax. This is better explain in the [`Matching Objects`](#matching-objects) section.

In [23]:
val = 1
match val:
    case 0:
        print("0")
    case True:
        print("true")
    case 1 | 2:
        print("1 or 2")
    case 3 | 4 as three_or_four:
        print("3 or 4", three_or_four)
    case int() as v if v < 100:
        print("guard: int less than 100", v)
    case int(v) if 100 <= v < 200:
        print("int(v) is a shorthand syntax for `int() as v` this is only applied to builtin types")
        print("guard: 1xx", v)
    case v if not v:
        print("Capture v for a truthy value", v)
    case None:
        print("Matched None")
    case _:
        print("Wildcard")

1 or 2


### Gotcha: Variable Value Matching

It's quite easy to confuse matching the value of a variable with the capture pattern

In [24]:
up = (0, 1)
match (1, 1):
    case up:  # this actually just means capture the value into a variable up
        print("Always matches")

Always matches


To get around this, your variable value expression must have a  `.` in there. This is commonly used for enums.

In [25]:
from enum import Enum, auto


class Direction:
    other = (1, 0)
    up = (0, 1)


match (0, 1):
    case Direction.up:
        print("Matched")


class Direction(Enum):
    UP = auto()


match Direction.UP:
    case Direction.UP:
        print("Matched")

Matched
Matched


Guard statements can also work though quite verbose:

In [26]:

up = (0, 1)
match (0, 1):
    case d if d == up:
        print("Matched")


Matched


## Matching Objects
The syntax is as follows
```py
match something:
    case ClassToMatch(attribute=expression):
        ...
```
where expression can be a any matching expression including nested.

The expression is equivalent to:
1. check `isinstance(something, ClassToMatch)`
2. match `something.attribute` with expression


In [27]:
from dataclasses import dataclass

@dataclass
class Link[T]:
    val: T
    link: "Link[T] | None" = None


links = Link(5, Link(4, Link(3, Link(2, Link(1, Link(0))))))
match links:
    case Link(val=1):
        print(
            "When matching object attributes "
            "there's no need to exhaustively match everything"
        )
    case object(val=1):
        print(
            "Since object is a super class of everything, "
            "this will match any type who has a `val` attribute of 1"
        )

    case object(blah=val):
        print("The object in question doesn't have attribute `blah")

    case Link(val=5, link=Link(val=3)):
        print("Nested match pattern")

    case Link(val=first, link=Link(val=4, link=third) as second):
        print("Nested capture pattern with a mixture of variable and as")
        print(first, second.val, third)


Nested capture pattern with a mixture of variable and as
5 4 Link(val=3, link=Link(val=2, link=Link(val=1, link=Link(val=0, link=None))))


### Positional Attribute Matching
In our previous examples, we're matching attributes using their name: 
```py
case Link(val=first, link=Link(val=4, link=third) as second):
```

This is quite verbose, we can actually specify order of attribute for positional matching via `__match_args__`.

In [28]:
class Pair[T]:
    __match_args__ = ("first", "second")

    def __init__(self, first: T, second: T) -> None:
        self.first = first
        self.second = second


match Pair(1, 3):
    case Pair(1, 2):
        print("Not quite matching")
    case Pair(1, second):
        print("Match first attribute capture second:", second)


Match first attribute capture second: 3


Dataclasses have `__match_args__` populated by default.

In [29]:
links = Link(5, Link(4, Link(3, Link(2, Link(1, Link(0))))))
match links:
    case Link(5, Link(4)):
        print("Match by positional args")


Match by positional args



## Sequence Matching Syntax
Definition of [Python sequences](https://docs.python.org/3/library/stdtypes.html#sequence-types-list-tuple-range) include the builtin types `str, list, tuple`. There are also custom sequence types that inherit `collections.abc.Sequence`. 

### Sequence matching syntax
The special syntax for matching sequences are specified [here](https://peps.python.org/pep-0634/#sequence-patterns). 

Note:
- There are no differences between types like `list`, `tuple` and subtypes of `Sequence`.
- Unlike other types of matching, all elements of a sequence must be exhaustively matched.
- Adding to that, `[]` `()` or no brackets behave the same.
- `str` is a special case here and can't be matched with the syntax.

In [30]:
seq = ["a", "b", "c"]

match seq:
    case ["a", *rest]:
        print("matched first and rest is:", rest)
    case _:
        print("Not matched")


print("round brackets are equivalent")
match seq:
    case ("a", *rest):
        print("matched first and rest is:", rest)
    case _:
        print("Not matched")

print("no brackets are the same too")
match seq:
    case "a", *rest:
        print("matched first and rest is:", rest)
    case ["a"] | []:
        print("Brackets are needed only when you match a sequence of 1 or 0")
    case _:
        print("Not matched")


print("A str is left out of the sequence party")
print("Matching 'abc'")
match "abc":
    case "a", *_:
        print("matched first")
    case ["a"] | []:
        print("Brackets are needed only when you match a sequence of 1 or 0")
    case _:
        print("Not matched")



matched first and rest is: ['b', 'c']
round brackets are equivalent
matched first and rest is: ['b', 'c']
no brackets are the same too
matched first and rest is: ['b', 'c']
A str is left out of the sequence party
Matching 'abc'
Not matched


#### What can be matched
According to the [PEP](https://peps.python.org/pep-0634/#sequence-patterns):
> The following standard library classes will have their Py_TPFLAGS_SEQUENCE bit set:
> 
> - array.array
> - collections.deque
> - list
> - memoryview
> - range
> - tuple

In addition to subclasses of `Sequence`

In [31]:
from collections import UserList
from collections.abc import Sequence

class LowerCaseList(UserList[str]):
    def __init__(self, data: Sequence[str] | None = None) -> None:
        if data is None:
            data = []
        self.data = data

    def __getitem__(self, key: int) -> str:
        return super().__getitem__(key).lower()
    

match LowerCaseList(["A", "B", "C"]):
    case "a", *rest:
        print("matched first and rest is:", rest)
    case _:
        print("Not matched")


matched first and rest is: ['b', 'c']


#### Matching the sequence type and structure
How do we specify that we only want to match a tuple not a list?

There's a special syntax for this, which also works for `int`, `float` and `bool`

In [32]:
match "a", 1:
    case list([a, b]):
        print("list:", a, b)
    case tuple([a, b]):
        print("tuple:", a, b)

tuple: a 1


## Mapping Matching
Mappings are treated more similar to objects than sequences
- Specific keys are expected to be matched
- Matching is non-exhaustive
- There is no wildcard options for matching

In [33]:
match {"a": 1, "b": 2}:
    case {"a": str(_)}:
        print("Matching a key as a string")
    case {"b": int(b)}:
        print("Matching b key as a int", b)

Matching b key as a int 2


### Custom matching
The custom mapping must be a subclass of `collections.abc.Mapping`.

Even though custom types that implement `__getitem__` behave similar to a Mapping, it cannot be matched with a mapping type.

In [34]:
from collections.abc import Mapping
from dataclasses import dataclass


@dataclass
class Shift(Mapping):
    n: int

    def __iter__(self):
        yield from ()

    def __len__(self) -> int:
        return 26 - self.n

    def __getitem__(self, key: str) -> str:
        return chr(ord(key) + self.n)


print("a shifted 5 is", Shift(5)["a"])
match Shift(5):
    case {"a": "f", "b": "g"}:
        print("Matched")

a shifted 5 is f
Matched


## Exhaustive Matching
Unlike matching in other languages such as rust and ocaml. Matching in Python does not have to be exhaustive. Meaning it is not required to handle all possible cases.

It is however often a good idea to in order to error when unexpected values come through:

In [35]:
from enum import Enum, auto


class MyEnum(Enum):
    A = auto()
    B = auto()


def translate(enum: MyEnum) -> int:
    match enum:
        case MyEnum.A:
            return 0
        case MyEnum.B:
            return 1
        case _:
            # NOTE: You may wish to use an actual exception.
            assert False, "An unexpected value was passed in, let's error"

print(translate(MyEnum.A))


0


#### Typing + Exhaustive Matching
The above example works fine, but we can actually leverage the type system to do better. This is best demonstrated step-by-step.

##### Use `assert_never` at the end
The last branch should match anything and be passed into [`assert_never`](https://docs.python.org/3/library/typing.html#typing.assert_never). 
What we're saying here is that the `other` branch should never happen, that is the matching is exhaustive.

Code sample in [pyright playground](https://pyright-play.net/?code=GYJw9gtgBApgdgV2gSwgBzCALlAooiAGigEMEswAoUSKLATzWTgHMpUNtSBnbmbAPpwYAN36UJAYwA2JXlACy9fEgAUKiAEoAXJSj6oAQSgBeUuTCrNegwCFT5ilYmUAJjGB0QJON1lYYVXgkbUVlAk0oAFoAPnY4LF0DKAgSLEkAC1gCJOSDSTkYKDAsDP5cvLy5PkFhMRBVErKQTSA)

```python
from enum import Enum, auto
from typing import assert_never


class MyEnum(Enum):
    A = auto()
    B = auto()


def translate(enum: MyEnum) -> int:
    match enum:
        case other:
            assert_never(other)  # ❌
```

#### Add branches until the type checker passes
Code sample in [pyright playground](https://pyright-play.net/?code=GYJw9gtgBApgdgV2gSwgBzCALlAooiAGigEMEswAoUSKLATzWTgHMpUNtSBnbmbAPpwYAN36UJAYwA2JXlACy9fEgAUKiAEoAXJSj6oAQSgBeUuTCrNegwCFT5ilYmUAJjGB0QJON1lYYVXgkbUVlAk0oAFoAPnY4LF0DKAgSLEkAC1gCJOSDSTkYMI0AOkNcvLyQGCwEEDgoAAYbSoK%2BYoIS2wrKg2ra%2BqgARha8tqKwLAz%2BHt6ePkFhMRBVSemQSKgAYihAUHIgA)

```python
from enum import Enum, auto
from typing import assert_never


class MyEnum(Enum):
    A = auto()
    B = auto()


def translate(enum: MyEnum) -> int:
    match enum:
        case MyEnum.A:
            return 0
        case MyEnum.B:
            return 1
        case other:
            assert_never(other)  # ✅
```


#### Protection from Change
This example is simple and the enum is located in the same place. In more complex project structures this type checking can protect us from future changes to type definitions. 

Any addition to enum will cause the `assert_never` to fail the type check.
Code sample in [pyright playground](https://pyright-play.net/?code=GYJw9gtgBApgdgV2gSwgBzCALlAooiAGigEMEswAoUSKLATzWTgHMpUNtSBnbmbAPpwYAN36UJAYwA2JXlACy9fEgAUKiAEoAXJSj6oAQSgBeUuTCrNegwCFT5ilZv6Awg7JPrEgCYxgdCAkcNyyWDCq8EjaisoEmlAAtAB87HBYugZQECRYkgAWsASZWQaScjCxGgB0hiWlpSAwWAggcFAADC6l5XxVBNW29Q0GTS1tUACM3Vm9lWBY%2BfzDIzx8gsJiIKoLSyAJUADEUIAy5EA)

```python
from enum import Enum, auto
from typing import assert_never


class MyEnum(Enum):
    A = auto()
    B = auto()
    C = auto()


def translate(enum: MyEnum) -> int:
    match enum:
        case MyEnum.A:
            return 0
        case MyEnum.B:
            return 1
        case other:
            assert_never(other)  # ❌
```


## Advanced
### Matching Properties
I use this trick in [pattern-utils](https://github.com/Jamie-Chang/pattern-utils), since we can match on attribute of an object we must be able to match on a `property` (or any `descriptor`):

In [36]:
from dataclasses import dataclass
from functools import cached_property
import math


@dataclass
class Pair:
    first: int
    second: int

    @property
    def manhattan(self) -> int:
        return self.first + self.second
    
    @cached_property
    def linear(self) -> float:
        return math.sqrt(self.first * self.first + self.second * self.second)
    

match Pair(3, 4):
    case Pair(manhattan=7, linear=5):
        print("matched")

matched
